In [24]:
import csv
import pandas as pd
import nltk
import random

data = pd.read_csv("sms-spam-collection-dataset/spam.csv", encoding='latin-1')

In [28]:
data = data[['v1', 'v2']] 

In [44]:
i=0
documents = []
while i < int(data.shape[0]):
    documents.append((data.iloc[i][0], data.iloc[i][1].split()))
    i+=1

In [87]:
#shuffling as to not get bias data
random.shuffle(documents)

In [88]:
every_word = []
for j in data["v2"]:
    j = j.lower()
    split_word= j.split()
    every_word.extend(split_word)


In [89]:
#Frequency distribution in NLTK to remove useless words
every_word=nltk.FreqDist(every_word)

In [110]:
word_features = [x[0] for x in every_word.most_common(3000)]


In [113]:
def find_features(d):
    words = set(d)
    features = {}
    for w in word_features:
        features[w]= (w in words)
    return features

In [114]:
features_set=[(category, find_features(rev)) for (category, rev) in documents]

In [115]:
training_set = features_set[1900:]
testing_set =features_set[:1900]

In [119]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("accuracy", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

TypeError: unhashable type: 'dict'

In [ ]:


#from sklearn import LogisticRegression
#lg = LogisticRegression()
#lg.fit(X_train, Y_train)
#Y_predicted = lg.predict(X_train)

def error(actual_y, predict_y):
    return err

train_error = error(Y_train, Y_predicted)